In [3]:
import netCDF4 as nc
import numpy as np
from salishsea_tools import evaltools as et, places, viz_tools, visualisations
import datetime as dt

In [4]:
def halo_de(ncname,ts_x,ts_y):
    
    ''' given a path to a SalishSeaCast netcdf file and an x, y pair, 
        returns halocline depth, where halocline depth is defined a midway between 
        two cells that have the largest salinity gradient
        ie max abs((sal1-sal2)/(depth1-depth2))

            Parameters:
                    ncname (str): path to a netcdf file containing 
                    a valid salinity variable (vosaline)
                    ts_x (int): x-coordinate at which halocline is calculated
                    tx_y (int): y-coordinate at which halocline is calculated
            Returns:
                    halocline_depth: depth in meters of maximum salinity gradient
    '''
    
     # o
    
    halocline = 0
    grid = nc.Dataset('/data/vdo/MEOPAR/NEMO-forcing/grid/mesh_mask201702.nc')
    nemo = nc.Dataset(ncname)
    
    #get the land mask
    col_mask = grid['tmask'][0,:,ts_y,ts_x] 
        # to eliminate from the calculation any land points
    
    #get the depths of the watercolumn and filter only cells that have water
    col_depths = grid['gdept_0'][0,:,ts_y,ts_x]
    col_depths = col_depths[col_mask==1]    

    ### if there is no water, no halocline
    if (len(col_depths) == 0):
        halocline = np.nan
    
    else: 
        #get the salinity of the point, again filtering for where water exists
        col_sal = nemo['vosaline'][0,:,ts_y,ts_x]
        col_sal = col_sal[col_mask==1] # col_sal has a vector of salinity points, 
            # this gives true values where col_mask=1

        #get the gradient in salinity
        sal_grad = np.zeros_like(col_sal)

        for i in range(0, (len(col_sal)-1)):
            sal_grad[i] = np.abs((col_sal[i]-col_sal[i+1])/(col_depths[i]-col_depths[i+1]))

        #print(sal_grad)

        loc_max = np.where(sal_grad == np.nanmax(sal_grad))
        loc_max = (loc_max[0][0])

        #halocline is halfway between the two cells
        halocline = col_depths[loc_max] + 0.5*(col_depths[loc_max+1]-col_depths[loc_max])

    
    return halocline

In [8]:
#testnc = '/results2/SalishSea/nowcast-green.201905/05jan20/SalishSea_1d_20200105_20200105_grid_T.nc'
ii = 258
ij = 450
iyear=2015
starti=dt.datetime(iyear,1,1)
endi=dt.datetime(iyear,1,4)
basedir='/results2/SalishSea/nowcast-green.201905/'
nam_fmt='nowcast'
flen=1 # files contain 1 day of data each
tres=24 # 1: hourly resolution; 24: daily resolution  
flist3 = et.index_model_files(starti,endi,basedir,nam_fmt,flen,"grid_T",tres)
testnc=flist3['paths'][0]

halocline=list()

for filedate in flist3['paths']:
    halocline.append(halo_de(filedate,ii,ij))
    
print(halocline)

ValueError: zero-dimensional arrays cannot be concatenated

In [12]:
testnc[0]

'/results2/SalishSea/nowcast-green.201905/01jan15/SalishSea_1d_20150101_20150101_grid_T.nc'

In [5]:

testnc = '/results2/SalishSea/nowcast-green.201905/01jan15/SalishSea_1d_20150101_20150101_grid_T.nc'
ts_x = 258
ts_y = 450
halo = halo_de(testnc,ts_x,ts_y)
print(halo)

8.000929594039917
